# Examples with CSV files

> **These are Go notebooks**: In order to use the GoNB Jupyter Kernel, please install GoNB from here: [https://github.com/janpfeifer/gonb](https://github.com/janpfeifer/gonb)

Note also that for local package development, you can put: `!*go mod edit -replace "github.com/umbralcalc/stochadex=/path/to/stochadex"` at the top of any cell.

## To CSV

Writing some stored data of a partition to CSV can be done by first converting to a `gota` dataframe and then calling its CSV writer method. Here's a simple example below using generated data from a Normal distribution.

In [ ]:
import (
	"os"

	"github.com/umbralcalc/stochadex/pkg/simulator"
	"github.com/umbralcalc/stochadex/pkg/analysis"
	"github.com/umbralcalc/stochadex/pkg/inference"
)

%%

// Create a simulator.StateTimeStorage from a simulation run
storage := analysis.NewStateTimeStorageFromPartitions(
	// Instantiate the desired simulation state partitions
	[]*simulator.PartitionConfig{
		{
			Name:              "generated_data",
			Iteration:         &inference.DataGenerationIteration{
				Likelihood: &inference.NormalLikelihoodDistribution{},
			},
			Params:            simulator.NewParams(map[string][]float64{
				"mean": {1.8, 5.0, -7.3, 2.2},
                "covariance_matrix": {2.5, 0.0, 0.0, 0.0, 0.0, 9.0, 0.0, 0.0, 0.0, 0.0, 4.4, 0.0, 0.0, 0.0, 0.0, 17.0},
			}),
			InitStateValues:   []float64{1.3, 8.3, -4.9, 1.6},
			StateHistoryDepth: 1,
			Seed:              1234567,
		},
    },
	// Decide when should we stop the simulation
	&simulator.NumberOfStepsTerminationCondition{
		MaxNumberOfSteps: 1000,
	},
	// Decide how time should evolve
	&simulator.ConstantTimestepFunction{
		Stepsize: 1.0,
	},
	// Input the initial time
	0.0,
)

// Retrieve a dataframe representing the data in one partition
df := analysis.GetDataFrameFromPartition(storage, "generated_data")

// Save the dataframe as a CSV for later
file, _ := os.Create("data/test_csv.csv")
df.WriteCSV(file)

// Display the dataframe
fmt.Println(df)

## From CSV

We can also read CSV data into storage by using a function in the analysis package. The example below does this, where we have used the CSV data written to a file above.

In [ ]:
import (
	"github.com/umbralcalc/stochadex/pkg/simulator"
	"github.com/umbralcalc/stochadex/pkg/analysis"
	
	gonb_echarts "github.com/janpfeifer/gonb-echarts"
)

%%

// Read in CSV data directly into a simulator.StateTimeStorage
storage, _ := analysis.NewStateTimeStorageFromCsv(
	"data/test_csv.csv",
	0,
	map[string][]int{"generated_data" : {1, 2, 3, 4}},
	true,
)

// Reference the plotting data for the x-axis
xRef := analysis.DataRef{Plotting: &analysis.DataPlotting{IsTime: true}}

// Reference the plotting data for the y-axis
yRefs := []analysis.DataRef{{PartitionName: "generated_data"}}

// Create a scatter plot from partitions in a simulator.StateTimeStorage
scatter := analysis.NewScatterPlotFromPartition(storage, xRef, yRefs)

// Display the plot in a Go notebook
gonb_echarts.Display(scatter, "width: 1024px; height:400px; background: white;")

## Likelihood comparison

We can configure rolling windowed likelihood comparisons with relatively few steps using the functions provided in the analysis package. In the example below, we create such a comparison between the data read from CSV above and a Normal distribution likelihood model.

In [ ]:
import (
	"github.com/umbralcalc/stochadex/pkg/kernels"
	"github.com/umbralcalc/stochadex/pkg/simulator"
	"github.com/umbralcalc/stochadex/pkg/analysis"
	
	gonb_echarts "github.com/janpfeifer/gonb-echarts"
)

%%

// Read in CSV data directly into a simulator.StateTimeStorage
storage, _ := analysis.NewStateTimeStorageFromCsv(
	"data/test_csv.csv",
	0,
	map[string][]int{"generated_data" : {1, 2, 3, 4}},
	true,
)

// Configure a partition for computing the exponentially-weighted rolling mean
meanPartition := analysis.NewVectorMeanPartition(
	analysis.AppliedAggregation{
		Name: "mean",
		Data: analysis.DataRef{PartitionName: "generated_data"},
		Kernel: &kernels.ExponentialIntegrationKernel{},
	}, 
	storage,
)

// Set the timescale for the exponential weighting
meanPartition.Params.Set("exponential_weighting_timescale", []float64{100.0})

// Run and add the mean partition to storage
storage = analysis.AddPartitionsToStateTimeStorage(
	storage,
	[]*simulator.PartitionConfig{meanPartition},
	map[string]int{"generated_data": 200},
)

// Configure a partition for computing the exponentially-weighted rolling variance
variancePartition := analysis.NewVectorVariancePartition(
	analysis.DataRef{PartitionName: "mean"},
	analysis.AppliedAggregation{
		Name: "variance",
		Data: analysis.DataRef{PartitionName: "generated_data"},
		Kernel: &kernels.ExponentialIntegrationKernel{},
		DefaultValue: 1.0,
	},
	storage,
)

// Set the timescale for the exponential weighting
variancePartition.Params.Set("exponential_weighting_timescale", []float64{100.0})

// Run and add the variance partition to storage
storage = analysis.AddPartitionsToStateTimeStorage(
	storage,
	[]*simulator.PartitionConfig{variancePartition},
	map[string]int{"mean": 1, "generated_data": 200},
)

// Configure a model where the mean and variance are updated by the rolling estimators
model := analysis.ParameterisedModel{
	Likelihood: &inference.NormalLikelihoodDistribution{},
	Params: simulator.NewParams(make(map[string][]float64)),
	ParamsFromUpstream: map[string]simulator.NamedUpstreamConfig{
		"mean": {Upstream: "mean"},
		"variance": {Upstream: "variance"},
	},
}

// Configure a partition for computing the rolling likelihood comparison against the data
comparisonPartition := analysis.NewLikelihoodComparisonPartition(
	analysis.AppliedLikelihoodComparison{
		Name:  "loglikelihood",
		Model: model,
		Data:  analysis.DataRef{PartitionName: "generated_data"},
		Window: analysis.WindowedPartitions{
			Data: []analysis.DataRef{
				{PartitionName: "generated_data"},
				{PartitionName: "mean"},
				{PartitionName: "variance"},
			},
			Depth: 200,
		},
	},
	storage,
)

// Run and add the likelihood comparison partition to storage
storage = analysis.AddPartitionsToStateTimeStorage(
	storage,
	[]*simulator.PartitionConfig{comparisonPartition},
	map[string]int{"mean": 200, "variance": 200, "generated_data": 200},
)

// Reference the plotting data for the x-axis
xRef := analysis.DataRef{
	Plotting: &analysis.DataPlotting{
		IsTime: true,
		TimeRange: &analysis.IndexRange{Lower: 200, Upper: 1000},
	},
}

// Reference the plotting data for the y-axis
yRefs := []analysis.DataRef{{
	PartitionName: "loglikelihood",
	ValueIndices: []int{12},
	Plotting: &analysis.DataPlotting{
		TimeRange: &analysis.IndexRange{Lower: 200, Upper: 1000},
	},
}}

// Create a line plot from partitions in a simulator.StateTimeStorage
line := analysis.NewLinePlotFromPartition(storage, xRef, yRefs, nil)

// Display the plot in a Go notebook
gonb_echarts.Display(line, "width: 1024px; height:400px; background: white;")

## Simulation inference

We can also configure a posterior estimation and sampling of simulation parameters given the rolling data models provided for likelihood comparison. In the example below, we do this to estimate the mean vector of the normal distribution for the data we wrote to CSV above.

In [ ]:
import (
	"github.com/umbralcalc/stochadex/pkg/kernels"
	"github.com/umbralcalc/stochadex/pkg/simulator"
	"github.com/umbralcalc/stochadex/pkg/analysis"
	
	gonb_echarts "github.com/janpfeifer/gonb-echarts"
)

%%

// Read in CSV data directly into a simulator.StateTimeStorage
storage, _ := analysis.NewStateTimeStorageFromCsv(
	"data/test_csv.csv",
	0,
	map[string][]int{"generated_data" : {1, 2, 3, 4}},
	true,
)

// Configure a partition for computing the exponentially-weighted rolling mean
meanPartition := analysis.NewVectorMeanPartition(
	analysis.AppliedAggregation{
		Name: "mean",
		Data: analysis.DataRef{PartitionName: "generated_data"},
		Kernel: &kernels.ConstantIntegrationKernel{},
	}, 
	storage,
)

// Set the timescale for the exponential weighting
meanPartition.Params.Set("exponential_weighting_timescale", []float64{100.0})

// Run and add the mean partition to storage
storage = analysis.AddPartitionsToStateTimeStorage(
	storage,
	[]*simulator.PartitionConfig{meanPartition},
	map[string]int{"generated_data": 200},
)

// Configure a partition for computing the exponentially-weighted rolling variance
variancePartition := analysis.NewVectorVariancePartition(
	analysis.DataRef{PartitionName: "mean"},
	analysis.AppliedAggregation{
		Name: "variance",
		Data: analysis.DataRef{PartitionName: "generated_data"},
		Kernel: &kernels.ConstantIntegrationKernel{},
		DefaultValue: 1.0,
	},
	storage,
)

// Set the timescale for the exponential weighting
variancePartition.Params.Set("exponential_weighting_timescale", []float64{100.0})

// Run and add the variance partition to storage
storage = analysis.AddPartitionsToStateTimeStorage(
	storage,
	[]*simulator.PartitionConfig{variancePartition},
	map[string]int{"mean": 1, "generated_data": 200},
)

// Configure a model where the mean and variance are updated by the rolling estimators
model := analysis.ParameterisedModel{
	Likelihood: &inference.NormalLikelihoodDistribution{},
	Params: simulator.NewParams(make(map[string][]float64)),
	ParamsFromUpstream: map[string]simulator.NamedUpstreamConfig{
		"mean": {Upstream: "mean"},
		"variance": {Upstream: "variance"},
	},
}

// Configure a 'simulation' to infer the posterior samples of target parameters
simulation := &simulator.PartitionConfig{
	Name:              "regenerated_data",
	Iteration:         &inference.DataGenerationIteration{
		Likelihood: &inference.NormalLikelihoodDistribution{},
	},
	Params:             simulator.NewParams(map[string][]float64{
		"covariance_matrix": {2.5, 0.0, 0.0, 0.0, 0.0, 9.0, 0.0, 0.0, 0.0, 0.0, 4.4, 0.0, 0.0, 0.0, 0.0, 17.0},
	}),
	InitStateValues:    []float64{0.0, 0.0, 0.0, 0.0},
	StateHistoryDepth:  1,
	Seed:               123,
}

// Configure some partitions which collectively estimate and sample from the posterior
// distribution over the target parameters (the rolling mean vector in this example)
partitions := analysis.NewPosteriorEstimationPartitions(
	analysis.AppliedPosteriorEstimation{
		Names: analysis.PosteriorEstimationNames{
			LogNorm:    "posterior_log_norm",
			Mean:       "posterior_mean",
			Covariance: "posterior_cov",
			Sampler:    "posterior_sampler",
		},
		Comparison: analysis.AppliedLikelihoodComparison{
			Name:  "loglikelihood",
			Model: model,
			Data:  analysis.DataRef{PartitionName: "regenerated_data"},
			Window: analysis.WindowedPartitions{
				Partitions: []analysis.WindowedPartition{{
					Partition: simulation,
					OutsideUpstreams: map[string]simulator.NamedUpstreamConfig{
						"mean": {Upstream: "posterior_sampler"},
					}, 
				}},
				Data: []analysis.DataRef{
					{PartitionName: "mean"},
					{PartitionName: "variance"},
				},
				Depth: 200,
			},
		},
		Defaults: analysis.PosteriorDefaults{
			LogNorm:    0.0,
			Mean:       []float64{0.0, 0.0, 0.0, 0.0},
			Covariance: []float64{5.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 5.0},
			Sampler:    []float64{0.0, 0.0, 0.0, 0.0},
		},
		PastDiscount: 1.0,
        MemoryDepth:  300,
		Seed:         1234,
	},
	storage,
)

// Run and add the likelihood comparison partition to storage
storage = analysis.AddPartitionsToStateTimeStorage(
	storage,
	partitions,
	map[string]int{"mean": 200, "variance": 200},
)

// Reference the plotting data for the x-axis
xRef := analysis.DataRef{Plotting: &analysis.DataPlotting{IsTime: true}}

// Reference the likelihood partition plotting data for the y-axis
yRefs := []analysis.DataRef{{
	PartitionName: "loglikelihood",
	ValueIndices: []int{12},
}}

// Create a line plot from partitions in a simulator.StateTimeStorage
line := analysis.NewLinePlotFromPartition(storage, xRef, yRefs, nil)

// Display the plot in a Go notebook
gonb_echarts.Display(line, "width: 1024px; height:400px; background: white;")

// Reference the posterior samples plotting data for the y-axis
yRefs = []analysis.DataRef{{PartitionName: "posterior_sampler"}}

// Create a scatter plot from partitions in a simulator.StateTimeStorage
scatter := analysis.NewScatterPlotFromPartition(storage, xRef, yRefs)

// Display the plot in a Go notebook
gonb_echarts.Display(scatter, "width: 1024px; height:400px; background: white;")

// Reference the rolling mean plotting data (the target params) for the y-axis
yRefs = []analysis.DataRef{{PartitionName: "mean"}}

// Create a line plot from partitions in a simulator.StateTimeStorage
line = analysis.NewLinePlotFromPartition(storage, xRef, yRefs, nil)

// Display the plot in a Go notebook
gonb_echarts.Display(line, "width: 1024px; height:400px; background: white;")